In [23]:
import sys
sys.path.append('../Module')

from loader_csv import charger_csv
from inpute_na import traiter_valeurs_manquantes
from duplicata_csv import verifier_doublons

# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)

# Maintenant, vous pouvez utiliser le DataFrame df comme vous le souhaitez
df.describe()


c:\Users\33679\OneDrive\Bureau\brief predict pret\Model_pret\Model\../Module\loader_csv.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_partie4 = pd.read_csv(partie4)


,LoanNr_ChkDgt,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural
count,8.991640e+05,899164.000000,899164.000000,899164.000000,899164.000000,899028.000000,899164.000000,899164.000000,899164.000000,899164.000000
mean,4.772612e+09,53804.391241,398660.950146,110.773078,11.411353,1.280404,8.430376,10.797257,2753.725933,0.757748
std,2.538175e+09,31184.159152,263318.312760,78.857305,74.108196,0.451750,236.688165,237.120600,12758.019136,0.646436
min,1.000014e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.589758e+09,27587.000000,235210.000000,60.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,4.361439e+09,55410.000000,445310.000000,84.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000
75%,6.904627e+09,83704.000000,561730.000000,120.000000,10.000000,2.000000,1.000000,4.000000,1.000000,1.000000
max,9.996003e+09,99999.000000,928120.000000,569.000000,9999.000000,2.000000,8800.000000,9500.000000,99999.000000,2.000000


In [24]:
df.shape

(899164, 27)

In [25]:
df.isna().sum()/df.count()

LoanNr_ChkDgt        0.000000
Name                 0.000016
City                 0.000033
State                0.000016
Zip                  0.000000
Bank                 0.001737
BankState            0.001745
NAICS                0.000000
ApprovalDate         0.000000
ApprovalFY           0.000000
Term                 0.000000
NoEmp                0.000000
NewExist             0.000151
CreateJob            0.000000
RetainedJob          0.000000
FranchiseCode        0.000000
UrbanRural           0.000000
RevLineCr            0.005061
LowDoc               0.002880
ChgOffDate           4.526549
DisbursementDate     0.002641
DisbursementGross    0.000000
BalanceGross         0.000000
MIS_Status           0.002226
ChgOffPrinGr         0.000000
GrAppv               0.000000
SBA_Appv             0.000000
dtype: float64

In [26]:
# Supposons que df est votre DataFrame
df_traite = traiter_valeurs_manquantes(df)
df.shape

(899164, 26)

In [27]:
df.isna().sum()/df.count()

LoanNr_ChkDgt        0.0
Name                 0.0
City                 0.0
State                0.0
Zip                  0.0
Bank                 0.0
BankState            0.0
NAICS                0.0
ApprovalDate         0.0
ApprovalFY           0.0
Term                 0.0
NoEmp                0.0
NewExist             0.0
CreateJob            0.0
RetainedJob          0.0
FranchiseCode        0.0
UrbanRural           0.0
RevLineCr            0.0
LowDoc               0.0
DisbursementDate     0.0
DisbursementGross    0.0
BalanceGross         0.0
MIS_Status           0.0
ChgOffPrinGr         0.0
GrAppv               0.0
SBA_Appv             0.0
dtype: float64

In [28]:
df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,0,N,Y,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,0,N,Y,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,0,N,N,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,0,N,Y,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,0,N,N,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [29]:
# Supposons que df est votre DataFrame
doublons, df_sans_doublons = verifier_doublons(df)
print(doublons)

{}


In [32]:
df.duplicated().sum()

0